### Set the current directory with the Location Yolo Files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Download YOLO Weight Files and Other Required files from https://www.kaggle.com/valentynsichkar/yolo-coco-data

In [ ]:
%cd /content/drive/MyDrive/Kaggle/YOLO\ weights
!wget https://pjreddie.com/media/files/yolov3.weights
# !wget https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg

/content/drive/MyDrive/Kaggle/YOLO weights
--2020-11-21 11:19:04--  https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘yolov3.cfg’

yolov3.cfg              [ <=>                ] 261.41K  --.-KB/s    in 0.03s   

2020-11-21 11:19:04 (9.03 MB/s) - ‘yolov3.cfg’ saved [267686]



Set the Path to the folder containing YOLO Weight Files

In [ ]:
model_path='/content/drive/MyDrive/Kaggle/YOLO weights'

### Set the Threshold for Confidence filter Weak Detections

In [ ]:
min_COnfidence=0.5
NMS_Thres=0.4

In [ ]:
import numpy as np
import cv2

### We Create a function to Detect Features we want, We can extract more features as well which are avilable in coco.names

In [ ]:
def detectFeatures(frame, net, ln, bicycleIndx=0, carIndx=0, motorbikeIndx=0, busIndx=0, truckIndx=0):
    #Getting Height, Width of the current frame
    (H,W)=frame.shape[:2]
    #We'll store out predicitons with it's details such as confidence, bounding box points in results
    results=[]
    #Creating a blob of current frame
    blob=cv2.dnn.blobFromImage(frame, 1/255.0,(416,416), swapRB=True, crop=False)
    #Giving the blob as input to our model to process this frame
    net.setInput(blob)
    #Getting the output of our Model which contains predictions
    layerOutputs=net.forward(ln)
    #We'll Stores the bounding box co-ordinates, confidences, ID which corresponds to object it is in these for our predictions
    boxes=[]
    confidences=[]
    classIDs=[]
    #Iterate though our outputs and each detection in each output
    for output in layerOutputs:
        for detection in output:
            #Extract Confidence for each detection and it's ID for item it corresponds to
            scores=detection[5:]
            classID=np.argmax(scores)
            confidence=scores[classID]
            #We just want Bicycles, Cars, Bikes, Trucks and Buses to be detected
            if (classID==bicycleIndx or classID==carIndx or classID==motorbikeIndx or classID==busIndx or classID==truckIndx) and confidence>min_COnfidence:
                #Get the boundary box co-ordinates for our Detectection
                box=detection[0:4]*np.array([W,H,W,H])
                (centerX, centerY, width, height)=box.astype('int')
                x=int(centerX-(width/2))
                y=int(centerY-(height/2))
                #Save these co-ordinates, confidence, ID of item number in the lists we initialized
                boxes.append([x,y,int(width),int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
    #Applying Non-Max Suppression to remove extra Bounding Boxes
    indxs=cv2.dnn.NMSBoxes(boxes, confidences, min_COnfidence, NMS_Thres)
    #Storing Final Detection's features in results
    if len(indxs)>0:
        for i in indxs.flatten():
            (x,y)=(boxes[i][0], boxes[i][1])
            (w,h)=(boxes[i][2], boxes[i][3])
            r=(confidences[i], (x,y,x+w,y+h),classIDs[i])
            results.append(r)
    return results

In [ ]:
import numpy as np
import imutils
import os
from PIL import Image

In [ ]:
labelsPath='/content/YOLO weights/coco.names'
labels=open(labelsPath).read().strip().split('\n')
weightPaths='/content/YOLO weights/yolov3.weights'
configPath='/content/YOLO weights/yolov3.cfg'

### Creating our model

In [ ]:
net=cv2.dnn.readNetFromDarknet(configPath, weightPaths)

In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate

I've also shared the code for Training of the Emergency Vehicles Classifier in this repo.

#### You can also download this Classifier's Weights files from here: 
https://drive.google.com/file/d/1-R6dkaTLXHB1Yyn1GH7LX9-Hpz5Mjx-z/view?usp=sharing

In [ ]:
emergencyModel=load_learner('/content/EmergencyVehicleClassificationFastAI','emergency_vehicles.pkl')

### Giving the Input Video to process and output Location

In [ ]:
%cd /content/
inp='/content/Transportation - Traffic Monitoring in a section (AI).mp4'
output='/content/output1.avi'
display=0

/content


In [ ]:
ln=net.getLayerNames()
ln=[ln[i[0]-1] for i in net.getUnconnectedOutLayers()]
vs=cv2.VideoCapture(inp)
writer=None
fps=vs.get(cv2.CAP_PROP_FPS)

In [ ]:
tempPath='/content/EmergencyVehicleClassificationFastAI/1.jpg'

In [ ]:
frameNumber=0
light=(0,0,255)
while True:
    #Reading frame from video
    (check, frame1) = vs.read()
    frameNumber+=1
    if int(frameNumber/fps)==17:
        print('Done 1')
        light=(0,255,0)
    elif int(frameNumber/fps)==20:
        light=(0,0,255)
    elif int(frameNumber/fps)==25:
        print('Stop')
        break
    if not check:
        break
    frame=frame1.copy()
    #Resizing Frame
    frame = imutils.resize(frame, width=700)
    #Extracting Results from the above function
    results = detectFeatures(frame, net, ln,bicycleIndx=labels.index('bicycle'), carIndx=labels.index('car'), motorbikeIndx=labels.index('motorbike'), busIndx=labels.index('bus'), truckIndx=labels.index('truck'))
    #Setting Number of Vehicles initially 0
    vehicles=0
    urgent=False
    #Iterating through our results in Current frame
    for (i,(prob, bbox, classID))in enumerate(results):
        #Getting co-ordinates our current detection, and adding to corresponding Detection we got
        (startX, startY, endX, endY)=bbox
        if classID==labels.index('car') or classID==labels.index('bus') or classID==labels.index('truck'):
            vehicles+=1
            color=(0,255,0)
            try:
                vehicle=frame[startY:endY, startX:endX]
                cv2.imwrite(tempPath,vehicle)
                temp=open_image('/content/drive/MyDrive/Kaggle/Projects/EmergencyVehicleClassificationFastAI/1.jpg')
                pred_class,pred_idx,outputs = emergencyModel.predict(temp)
                if str(pred_class)=='1':
                    color=(0,0,255)
                    urgent=True
                !rm -r '$tempPath'
            except:
                pass
        elif classID==labels.index('bicycle') or classID==labels.index('motorbike'):
            vehicles+=1
            color=(0,255,0)
        #Plotting rectangle around the detection and displaying what it is
        cv2.rectangle(frame, (startX,startY),(endX,endY),color,1)
        cv2.putText(frame, labels[classID], (startX,startY-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,color,1)
    #Displaying Summary of traffic on top-left
    text1='Vehicles: {}'.format(vehicles)
    text2='Emergency Vehicles: {}'.format(urgent)
    cv2.putText(frame, text1, (25,frame.shape[0]-350),cv2.FONT_HERSHEY_SIMPLEX,0.6, (0,0,255),2)
    cv2.putText(frame, text2, (25,frame.shape[0]-325),cv2.FONT_HERSHEY_SIMPLEX,0.6, (0,0,255),2)
    cv2.rectangle(frame, (555,30),(640,90),(255,0,0), -1)
    cv2.circle(frame, (600,60), 25, (0,0,255),-1)

    #If display>0, you'll get to see each frame while it's being processed and written to Output file
    if display>0:
        cv2.imshow('Frame',frame)
        key=cv2.waitKey(0)
        if key==27:
            break
        cv2.destroyAllWindows()
    if output!='' and writer is None:
        fourcc=cv2.VideoWriter_fourcc(*'MJPG')
        writer=cv2.VideoWriter(output, fourcc, fps, (frame.shape[1],frame.shape[0]))
    if writer is not None:
        writer.write(frame)
writer.release()

Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Done 1
Stop
